## Random Maze Environment

#### Learning the Maze

- Previously we created `FrozenPond` and then a `Maze` variant.
- Let's train an agent to complete the Maze using RLlib.

In [1]:
from ray.rllib.agents.ppo import PPOTrainer
from envs import Maze # Maze defined in previous slides

trainer = PPOTrainer({"framework" : "torch"}, env=Maze)

In [2]:
trainer.train()

In [6]:
from ipywidgets import Output
from IPython import display
import time

env = Maze()

out = Output();
display.display(out);
with out:
    obs = env.reset()
    env.seed(1)
    done = False
    while not done:
        action = trainer.compute_single_action(obs, explore=False)
        obs, rewards, done, _ = env.step(action)
        
        out.clear_output(wait=True)
        env.render()
        time.sleep(0.5)

....
.X.X
...X
X..G
